1：查看项目中Paddle、PaddleHub版本及模型信息（20分）

In [1]:
!pip install --upgrade paddlehub -i https://mirror.baidu.com/pypi/simple
!pip install --upgrade paddlepaddle -i https://mirror.baidu.com/pypi/simple
import paddle
print("paddle version:",paddle.__version__)
import paddlehub as hub
print("hub version:",hub.__version__)
model = hub.Module(name='resnet50_vd_animals')
print("model version:",model.version)



Looking in indexes: https://mirror.baidu.com/pypi/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://mirror.baidu.com/pypi/simple
  Using cached https://mirror.baidu.com/pypi/packages/e4/42/3a91bff89038a9773b2df69610293e759ed6e1de6fe115503d1132805103/paddlepaddle-2.2.2-cp37-cp37m-manylinux1_x86_64.whl (108.4 MB)
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
paddle version: 2.2.2
hub version: 2.2.0


[2022-03-29 11:00:42,198] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


model version: 1.0.0


W0329 11:00:42.202107  3232 analysis_predictor.cc:1350] Deprecated. Please use CreatePredictor instead.


2： 完成一件预测任务

In [2]:
# 预测任务
import cv2
classifier = hub.Module(name="resnet50_vd_animals")
result = classifier.classification(images=[cv2.imread('./fd.jpeg')])
print("图中的动物是：",result)


[2022-03-29 11:00:42,644] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


图中的动物是： [{'法兰西斗牛犬': 0.891738772392273}]


3 ：完成1个迁移学习任务（30分）​​

In [3]:
# 数据增强
import paddlehub.vision.transforms as T

transforms = T.Compose([T.Resize((256, 256)),
                        T.CenterCrop(224),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])],
                        to_rgb=True)
                        

In [4]:
from paddlehub.datasets import Flowers
from paddlehub.finetune.trainer import Trainer

flowers = Flowers(transforms)

flowers_validate = Flowers(transforms, mode='val')


In [5]:
model = hub.Module(name="resnet50_vd_imagenet_ssld", label_list=["roses", "tulips", "daisy", "sunflowers", "dandelion"])


load pretrained parameters success


In [6]:
optimizer = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
trainer = Trainer(model, optimizer, checkpoint_dir='img_classification_ckpt')


trainer.train(flowers, epochs=1, batch_size=32, eval_dataset=flowers_validate, save_interval=1)

[2022-03-29 11:00:45,768] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...
[2022-03-29 11:04:05,578] [   TRAIN] - Epoch=1/1, Step=10/92 loss=1.3130 acc=0.5219 lr=0.001000 step/sec=0.05 | ETA 00:30:38
[2022-03-29 11:07:19,584] [   TRAIN] - Epoch=1/1, Step=20/92 loss=1.0820 acc=0.5500 lr=0.001000 step/sec=0.05 | ETA 00:30:11
[2022-03-29 11:10:34,169] [   TRAIN] - Epoch=1/1, Step=30/92 loss=0.8142 acc=0.7031 lr=0.001000 step/sec=0.05 | ETA 00:30:04
[2022-03-29 11:13:49,498] [   TRAIN] - Epoch=1/1, Step=40/92 loss=0.7974 acc=0.7312 lr=0.001000 step/sec=0.05 | ETA 00:30:02
[2022-03-29 11:17:03,748] [   TRAIN] - Epoch=1/1, Step=50/92 loss=0.7971 acc=0.7188 lr=0.001000 step/sec=0.05 | ETA 00:29:59
[2022-03-29 11:20:18,363] [   TRAIN] - Epoch=1/1, Step=60/92 loss=0.7909 acc=0.7219 lr=0.001000 step/sec=0.05 | ETA 00:29:57
[2022-03-29 11:23:31,746] [   TRAIN] - Epoch=1/1, Step=70/92 loss=0.6773 acc=0.7500 lr=0.001000 step/sec=0.05 | ETA 00:29:55
[2022-03-29 11:26:45,667] 

In [8]:
#在线预测过程见pre.ipynb
# import paddle
# import paddlehub as hub

# model = hub.Module(name='resnet50_vd_imagenet_ssld', label_list=["roses", "tulips", "daisy", "sunflowers", "dandelion"], load_checkpoint='img_classification_ckpt/best_model/model.pdopt')
# result = model.predict(['fa.jpg'])
# print(result)


ValueError: parameter name [conv1_1_weights] have be been used. In dygraph mode, the name of parameter can't be same.Please check the parameter attr value passed to self.create_parameter or constructor of dygraph Layers

4：完成1个demo服务化部署任务（30分）​​

In [ ]:
!hub serving start -m resnet50_vd_imagenet_ssld


load pretrained parameters success
 * Serving Flask app "paddlehub.serving.app_compat" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8866/ (Press CTRL+C to quit)


In [ ]:


import requests
import json
import cv2
import base64

import numpy as np


def cv2_to_base64(image):
    data = cv2.imencode('.jpg', image)[1]
    return base64.b64encode(data.tostring()).decode('utf8')

def base64_to_cv2(b64str):
    data = base64.b64decode(b64str.encode('utf8'))
    data = np.fromstring(data, np.uint8)
    data = cv2.imdecode(data, cv2.IMREAD_COLOR)
    return data

# 发送HTTP请求
org_im = cv2.imread('fa.jpg')

data = {'images':[cv2_to_base64(org_im)], 'top_k':1}
headers = {"Content-type": "application/json"}
url = "http://127.0.0.1:8866/predict/resnet50_vd_imagenet_ssld"
r = requests.post(url=url, headers=headers, data=json.dumps(data))
data =r.json()["results"]['data']
print(data)


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 